In [2]:
import pandas as pd
import numpy as np
import glob
import os


In [3]:
data_root = 'data/TrainingData/'
X_TIME = 'subject_{}_{}__x_time.csv'
X_DATA = 'subject_{}_{}__x.csv'

Y_TIME = 'subject_{}_{}__y_time.csv'
Y_DATA = 'subject_{}_{}__y.csv'

In [4]:
files_x_data = sorted(glob.glob(data_root+X_DATA.format('*', '*')))
files_x_time = sorted(glob.glob(data_root+X_TIME.format('*', '*')))

files_y_data = sorted(glob.glob(data_root+Y_DATA.format('*', '*')))
files_y_time = sorted(glob.glob(data_root+Y_TIME.format('*', '*')))

In [5]:
pd_x = pd.DataFrame()
pd_y = pd.DataFrame()

In [6]:
for i in range(len(files_x_data)):
    y_data = pd.read_csv(files_y_data[i], names=['label'])
    y_time = pd.read_csv(files_y_time[i], names=['time'])

    x_data = pd.read_csv(files_x_data[i], names=['accr_x', 'accr_y','accr_z','gyr_x','gyr_y','gyr_z'])
    x_time = pd.read_csv(files_x_time[i], names=['time'])

    x_data = pd.concat([x_data, x_time], axis=1)
    y_data = pd.concat([y_data, y_time], axis=1)
    # downsampling here -----> 
    pd_x = pd.concat([pd_x, x_data], axis=0)
    pd_y = pd.concat([pd_y, y_data], axis=0)